# Provision a GPU Server on Chameleon and Connect to Colab

This notebook describes how to connect Colab to a server running on Chameleon. This allows you to run experiments requiring bare metal access, storage, memory, GPU and compute that exceeds the abilities of Colab's hosted runtime, but with Colab's familiar interface (and notebooks stored in your Google Drive). It also allows you to easily go back and forth between the convenience of Colab's hosted runtime and Chameleon's greater capabilities, depending on the needs of your experiment.

## Provision the resource


### Check resource availability

This notebook will try to reserve a bare metal Ubuntu server with RTX6000 GPU on CHI@UC - pending availability. Before you begin, you should check the host calendar at [https://chi.uc.chameleoncloud.org/project/leases/calendar/host/](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/). In the "Node Type" dropdown, filter on `gpu_rtx_6000` for GPU or `skylake` for CPU and make sure some hosts are available.

### Chameleon configuration

You can change your Chameleon project name (if not using the one that is automatically configured in the JupyterHub environment) and the site on which to reserve resources (depending on availability) in the following cell.

In [1]:
import chi, os, time
from chi import lease
from chi import server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME') # change this if you need to
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)
username = os.getenv('USER') # all exp resources will have this prefix

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


If you need to change the details of the Chameleon server, e.g. use a different GPU type depending on availability, you can do that in the following cell.

In [24]:
server_type="cpu" #for cpu
# server="gpu" #for gpu

In [25]:
if server_type=="cpu":
    NODE_TYPE="compute_skylake"
else:
    NODE_TYPE="gpu_rtx_6000"

### Reservation

The following cell will create a reservation that begins now, and ends in 8 hours. You can modify the start and end date as needed. 

If there is no compatible device available right now, you can refer to the [CHI@UC host calendar](https://chi.uc.chameleoncloud.org/project/leases/calendar/host/) (change the "Node type" selection to `gpu_rtx_6000`). Set the start and end time according to availability. Then,

* run the cell below to reserve your GPU node for some time in the future
* at the beginning of your (future) reservation, you'll run this notebook again but you will *skip* the "Reservation" section

In [4]:
res = []
lease.add_node_reservation(res, node_type=NODE_TYPE, count=1)

start_date, end_date = lease.lease_duration(days=2, hours=0)
# if you won't start right now - comment the line above, uncomment two lines below
# start_date = "2023-01-01 00:00" # manually define to desired start time 
# end_date =   "2023-01-01 08:00" # manually define to desired start time 

l = lease.create_lease(f"colab-{username}-{NODE_TYPE}-v1", res, start_date=start_date, end_date=end_date)
l = lease.wait_for_active(l["id"])

### Provisioning resources

This section provisions resources. It will take approximately 10 minutes. You can check on its status in the Chameleon web-based UI: [https://chi.uc.chameleoncloud.org/project/instances/](https://chi.uc.chameleoncloud.org/project/instances/), then come back here when it is in the READY state.

In [5]:
# continue here, whether using a lease created just now or one created earlier
# l = lease.get_lease(f"colab-{username}-{NODE_TYPE}-v1")
# reservation_id = lease.get_node_reservation(l["id"])
# server_id = server.get_server_id(f"colab-{username}-{NODE_TYPE}-v1")
# server.wait_for_active(server_id)

In [8]:
reservation_id = lease.get_node_reservation(l["id"])
server.create_server(
    f"colab-{username}-{NODE_TYPE}-v1", 
    reservation_id=reservation_id,
    image_name="CC-Ubuntu22.04"
)
server_id = server.get_server_id(f"colab-{username}-{NODE_TYPE}-v1")
server.wait_for_active(server_id)

openstack.compute.v2.server.Server(id=1377a136-83bf-4cb3-90cf-cf47383a0417, name=colab-sj4020_nyu_edu-compute_skylake-v1, status=ACTIVE, tenant_id=c819e73b9bfd4c0595bf49dcf338d51b, user_id=ce8013cadca94eee50231ad7e3c4c8e8ff82d4a89c7ddaf8ab430fd8703a8249, metadata={}, hostId=7cbd4d17ddcda48102257306b0d7d2bcee03b631dcd5ebd809bc0fff, image={'id': '9fd6526b-24c3-4f7d-a9d3-08135bbbb5b6', 'links': [{'rel': 'bookmark', 'href': 'https://chi.uc.chameleoncloud.org:8774/images/9fd6526b-24c3-4f7d-a9d3-08135bbbb5b6'}]}, flavor={'vcpus': 1, 'ram': 1, 'disk': 20, 'ephemeral': 0, 'swap': 0, 'original_name': 'baremetal', 'extra_specs': {'resources:CUSTOM_BAREMETAL': '1', 'resources:VCPU': '0', 'resources:MEMORY_MB': '0', 'resources:DISK_GB': '0'}}, created=2024-04-03T00:49:18Z, updated=2024-04-03T00:55:51Z, addresses={'sharednet1': [{'version': 4, 'addr': '10.140.82.185', 'OS-EXT-IPS:type': 'fixed', 'OS-EXT-IPS-MAC:mac_addr': '24:6e:96:7d:84:2e'}]}, accessIPv4=, accessIPv6=, links=[{'rel': 'self', 'hre

Associate an IP address with this server:

In [10]:
reserved_fip = server.associate_floating_ip(server_id)

and wait for it to come up:

In [11]:
server.wait_for_tcp(reserved_fip, port=22)

## Install stuff

The following cells will install some basic packages in order to connect your Colab frontend to your Chameleon server. However, you may want to log in to your Chameleon server in order to access its terminal and install or configure packages outside of Colab. There is an additional section called **GPU installations** which is supposed to be executed only if GPU server is initiated.

To log in to the resource, use File > New > Terminal in the Chameleon JupyterHub environment, or your local terminal, and run:


In [12]:
print("ssh cc@" + reserved_fip)

ssh cc@192.5.87.112


Meanwhile, install an updated CUDA, Python and JupyterHub on your resource:

In [28]:
from chi import ssh

node = ssh.Remote(reserved_fip)

In [ ]:
node.run('sudo apt update')
node.run('sudo apt -y install python3-pip python3-dev')
node.run('sudo pip3 install --upgrade pip')

### GPU installations

In [17]:
if server_type=="gpu":
    node.run('wget https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64/cuda-keyring_1.0-1_all.deb')
    node.run('sudo dpkg -i cuda-keyring_1.0-1_all.deb')
    node.run('sudo apt update')
    node.run('sudo apt -y install linux-headers-$(uname -r)')
    node.run('sudo apt -y install nvidia-driver-545') 
    try:
        node.run('sudo reboot') # reboot and wait for it to come up
    except:
        pass
    server.wait_for_tcp(reserved_fip, port=22)

In [18]:

if server_type=="gpu":
    node = ssh.Remote(reserved_fip) 
    node.run('sudo apt -y install cuda-12-2 cuda-runtime-12-2 cuda-drivers=545.23.08-1')
    node.run('sudo apt -y install nvidia-gds-12-2') # install instructions say to do this separately!
    node.run('sudo apt -y install libcudnn8=8.9.6.50-1+cuda12.2 cuda-toolkit-12-2') # make sure the get cuda-11-8 version
    node.run("echo 'PATH=\"/usr/local/cuda-12.2/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/snap/bin\"' | sudo tee /etc/environment")

Now we have to reboot, and make sure we have the specified CUDA:

In [21]:
if server_type=="gpu":
    try:
        node.run('sudo reboot')
    except:
        pass
    server.wait_for_tcp(reserved_fip, port=22)

In [19]:
if server_type=="gpu":
    node = ssh.Remote(reserved_fip) # note: need a new SSH session to get new PATH
    node.run('nvidia-smi')
    node.run('nvcc --version')

### Add SSH Keys to the new server

Finally, add all SSH keys in your account at this site - 

In [16]:
nova=chi.clients.nova()
# iterate over all keypairs in this account
for kp in nova.keypairs.list(): 
    public_key = nova.keypairs.get(kp.name).public_key 
    node.run(f"echo {public_key} >> ~/.ssh/authorized_keys")

#### Optional: Install Python packages

Before starting your Jupyter instance, you may want to install some Python packages.

The following cell will install the *same* version of some key deep learning packages as are installed on Colab (as of November 2022), for maximum cross-compatibility. 

If you need additional packages, you can install them in a similar manner.

### Set up Jupyter on server

Install `jupyter_http_over_ws`, which is required in order to connect Colab to this Jupyter instance:

In [ ]:
node.run('python3 -m pip install jupyter-core==5.5.0 jupyter-client==6.1.12 jupyter-server==1.24.0 jupyterlab-widgets==3.0.9  jupyterlab==3.6.6 jupyter_http_over_ws traitlets')

And, activate `jupyter_http_over_ws` - the output should show `jupyter_http_over_ws 0.0.7 OK` (don't worry if it also says "Validation failed"):

In [21]:
node.run('/home/cc/.local/bin/jupyter server extension enable --py jupyter_http_over_ws')

Enabling: jupyter_http_over_ws
- Writing config: /usr/etc/jupyter
    - Validating jupyter_http_over_ws...
      jupyter_http_over_ws 0.0.7 OK
      X Validation failed: [Errno 13] Permission denied: '/usr/etc'


<Result cmd='/home/cc/.local/bin/jupyter server extension enable --py jupyter_http_over_ws' exited=0>

In [14]:
node.run('python3 -m pip install -r requirements.txt')

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


<Result cmd='python3 -m pip install -r requirements.txt' exited=0>

## Connect Colab to the server

In a **local terminal on your own laptop**, run

In [22]:
print('ssh -L 127.0.0.1:8889:127.0.0.1:8889 cc@' + reserved_fip) 

ssh -L 127.0.0.1:8889:127.0.0.1:8889 cc@192.5.87.112


to set up a tunnel to the Jupyter server.

 If your Chameleon key is not in the default location, you should also specify the path to your key as an argument, using `-i`. For example:

In [23]:
print('ssh -i ~/.ssh/id_rsa_chameleon -L 127.0.0.1:8889:127.0.0.1:8889 cc@' + reserved_fip) 

ssh -i ~/.ssh/id_rsa_chameleon -L 127.0.0.1:8889:127.0.0.1:8889 cc@192.5.87.112


Leave this SSH session open.

Then, run the following cell, which will run a command that does not terminate: 

In [ ]:
node.run("/home/cc/.local/bin/jupyter notebook --NotebookApp.allow_origin='https://colab.research.google.com' --port=8889 --NotebookApp.port_retries=0")

[W 02:17:06.037 NotebookApp] Loading JupyterLab as a classic notebook (v6) extension.
[W 2024-04-03 02:17:06.039 LabApp] 'allow_origin' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2024-04-03 02:17:06.039 LabApp] 'port' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2024-04-03 02:17:06.039 LabApp] 'port_retries' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[W 2024-04-03 02:17:06.039 LabApp] 'port_retries' has moved from NotebookApp to ServerApp. This config will be passed to ServerApp. Be sure to update your config before our next release.
[I 2024-04-03 02:17:06.042 LabApp] JupyterLab extension loaded from /home/cc/.local/lib/python3.10/site-packages/jupyterlab
[I 2024-04-03 02:17:06.042 LabApp] JupyterLab appl

In the output of the cell above, look for a URL in this format:
    
```
http://localhost:8888/?token=XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX
```

Copy this URL - you will need it in the next step.

Now, you can open Colab in a browser. Click on the drop-down menu for "Connect" in the top right and select "Connect to a local runtime". Paste the URL you copied earlier into the space and click "Connect". Your notebook should now be running on your Colab host (you can put `!hostname` in a cell and run it to verify!)


## Release resources

If you finish with your experimentation before your lease expires,release your resources and tear down your environment by running the following (commented out to prevent accidental deletions).

This section is designed to work as a "standalone" portion - you can come back to this notebook, ignore the top part, and just run this section to delete your reasources.

In [28]:
# setup environment - if you made any changes in the top part, make the same changes here
import chi, os
from chi import lease, server

PROJECT_NAME = os.getenv('OS_PROJECT_NAME')
chi.use_site("CHI@UC")
chi.set("project_name", PROJECT_NAME)
username = os.getenv('USER') # all exp resources will have this prefix

lease = chi.lease.get_lease(f"colab-{username}-{NODE_TYPE}-v1")

Now using CHI@UC:
URL: https://chi.uc.chameleoncloud.org
Location: Argonne National Laboratory, Lemont, Illinois, USA
Support contact: help@chameleoncloud.org


ValueError: No leases found for name colab-sj4020_nyu_edu-gpu_rtx_6000

In [68]:
DELETE = False
# DELETE = True  # un-comment if you're ready to delete resources

if DELETE:

    # get info about resources
    server_id = chi.server.get_server_id(f"colab-{username}-{NODE_TYPE}-v1")
    reserved_fip = [d['addr'] for d in chi.server.show_server(server_id).addresses['sharednet1'] if d['OS-EXT-IPS:type']=='floating'][0]

    # delete server   
    chi.server.delete_server(server_id)

    # # release floating IP
    reserved_fip =  chi.lease.get_reserved_floating_ips(lease["id"])
    ip_info = chi.network.get_floating_ip(reserved_fip)
    chi.neutron().delete_floatingip(ip_info["id"])

    # delete lease
    chi.lease.delete_lease(lease["id"])


ValueError: No matching servers found for name "colab-sj4020_nyu_edu-gpu_rtx_6000-v2"